In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [ ]:
def db_connection(user, password, host, port,db):
    engine= psycopg2.connect(
         user= user,
         password= password,
         host= host,
         port=port,
         database= db
    )
    return engine

In [ ]:
db_engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='bronze')

In [ ]:
def execute_query(query):
    df = pd.read_sql(query, db_engine, index_col='Id')
    return df


In [ ]:
sql = '''
    SELECT * 
    FROM raw_data
    '''
raw_data= execute_query(sql).copy()
raw_data.head(5)


In [ ]:
df= raw_data

In [ ]:
raw_data.dtypes

In [ ]:
raw_data.isna().sum()

# Starting Data Cleaning

### Check for the data dublicates

In [ ]:
where_desc_is_null= raw_data[raw_data['Description'].isna()]
where_desc_is_null


In [ ]:
# Assuming raw_data is your DataFrame
rows_with_null_customer_id = raw_data[raw_data['CustomerID'].isnull()]
rows_with_null_customer_id


### When Descruption is null, the CustomerID is too null, so I decied to drop nulls 

In [ ]:
raw_data = raw_data[~raw_data['Description'].isna()]
raw_data.shape

### Explore the nulls in Customer id to make descision about it

In [ ]:
raw_data.isna().sum()

### As it's mentioned, when customer id is null the whole colunmns too is null,
### so i will drop the nulls in customerid where all columns is null too.

In [ ]:
#condition= raw_data['CustomerID'].isna() and raw_data['InvoiceDate'].isna() and raw_data['Description'].isna() and raw_data['Quantity'].isna()"
customerNa= df[df['CustomerID'].isna()]
customerNa.isna().sum()

### There is no relation between when customer is null and any other column, so we can drop the nulls now

In [ ]:
raw_data= raw_data[~raw_data['CustomerID'].isna()]
raw_data.shape

In [ ]:
raw_data.head()

In [ ]:
raw_data.isna().sum()

In [ ]:
dup_rows= raw_data[raw_data.duplicated()]
print(len(dup_rows))



In [ ]:
# Assuming raw_data is your DataFrame
num_duplicates = raw_data.duplicated().sum()

print("Number of duplicate rows:", num_duplicates)


In [ ]:
raw_data.shape

In [ ]:
raw_data= raw_data.drop_duplicates()

In [ ]:
# Assuming raw_data is your DataFrame
duplicated_invoice_numbers = raw_data[raw_data.duplicated(subset=['InvoiceNo'], keep=False)]

duplicated_invoice_numbers.head()


In [ ]:
raw_data['saleskey']= raw_data['InvoiceNo'].astype(str) + raw_data['StockCode']


In [ ]:
dup_saleskey= raw_data[raw_data.duplicated(subset=['saleskey'])].sort_values('saleskey')
dup_saleskey.where(dup_saleskey['saleskey'] == '53641222940' )


In [ ]:
dup_rows= raw_data[raw_data['saleskey'].duplicated()]
dup_rows

### Handling the outliers in Quantity Col

In [ ]:
max_val=df['Quantity'].max()
max_val

In [ ]:
min_val= df['Quantity'].min()
min_val

### No quantity in negative, so i will convert all negative values

In [ ]:
df.loc[df['Quantity'] <0]


In [ ]:
df.loc[df['Quantity'] < 0, 'Quantity'] = df.loc[df['Quantity'] < 0, 'Quantity'] * -1
df.shape

In [ ]:
min_val=df['Quantity'].min()
min_val

In [ ]:
first_quartile =df['Quantity'].quantile(0.25)
first_quartile

In [ ]:
third_quartile =df['Quantity'].quantile(0.75)
third_quartile

In [ ]:
IQR= third_quartile - first_quartile
IQR

In [ ]:
df['Quantity'].value_counts()

In [ ]:
# max = IQR*10+Q3 threshold here is 10
max_outlier= IQR*10+third_quartile
max_outlier

In [ ]:
df.loc[df['Quantity'] >max_outlier].count()

In [ ]:
df.loc[df['Quantity'] >max_outlier] = max_outlier

In [ ]:
df.loc[df['Quantity'] <max_outlier]['Quantity'].max()

In [ ]:
df['Quantity'].plot(kind='box')

### Check Outliers in price

In [ ]:
df.shape

In [ ]:
df['UnitPrice'].value_counts()

In [ ]:
df.loc[df['UnitPrice'] > 190].count()

In [ ]:
price_max_val= df['UnitPrice'].max()
price_max_val

In [ ]:
price_min_val= df['UnitPrice'].min()
price_min_val

In [ ]:
Q1= df['UnitPrice'].quantile(0.25)
Q3= df['UnitPrice'].quantile(0.75)
IQR= Q3-Q1
print("Q1:",Q1)
print("Q2:",Q3)
print("IQR:",IQR)
max_outlier= IQR*1.5+Q3
print("max outlier value", max_outlier)

In [ ]:
df['UnitPrice'].plot(kind='box')

In [ ]:
df.loc[df['UnitPrice'] >750].count()

In [ ]:
df.loc[df['UnitPrice'] >750]= 750

In [ ]:
df['UnitPrice'].plot(kind='box')

### Now Data Cleaned, not have Duplicates or Outliers. and ready to be staged in the Database

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['CustomerID']= df['CustomerID'].astype(int)
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])

In [ ]:
df.dtypes

In [ ]:
df.head(5)

In [ ]:
def db_connection(user, password, host, port,db):
    engine= create_engine(
         f'postgresql://{user}:{password}@{host}:{port}/{db}'
    )
    engine.connect()
    return engine

In [ ]:
engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='bronze')

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head(n=0).to_sql(name='retail_cleaned', con=engine, if_exists='replace', index=True, index_label='Id')

In [ ]:
df.shape

In [ ]:
df.to_sql(name='retail_cleaned', con=engine, if_exists='append', index=True, index_label='Id')